# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 3/20/18 <br>
**File** numclean.ipynb <br>
**Purpose** Formating numerical (Bloomberg) data

<br><br>

In [5]:
# Imports
import numpy as np
import pandas as pd
import datetime
import matplotlib.pylab as plt
%matplotlib inline 

In [6]:
def getDates(rowlist, dates):
    datestring = rowlist[1]
    datelist = datestring.split('/')
    year = int(datelist[2])
    if (int(year) > 20):
        year += 1900
    else:
        year += 2000
    datelist[2] = str(year)
    old_datelist = datelist
    datelist = ''
    for j in range(0,len(old_datelist)):
        datelist += old_datelist[j]
        if j < len(old_datelist)-1:
            datelist += '/'
    dateobject = datetime.datetime.strptime(datelist,'%m/%d/%Y')
    dates.append(dateobject)    
    return dates

def getVols(rowlist, vols):
    
    volstring = rowlist[3]
    
    vol = 0
    
    if ('M' in volstring) or ('m' in volstring):
        if 'M' in volstring:
            vollist = volstring.split('M')
        else:
            vollist = volstring.split('m')
        vol = float(vollist[0])
        vol *= 1000000
        vols.append(vol)
        return vols
    elif ('K' in volstring) or ('k' in volstring):
        if 'K' in volstring:
            vollist = volstring.split('K')
        else:
            vollist = volstring.split('k')
        vol = float(vollist[0])
        vol *= 1000
        vols.append(vol)
        return vols
    elif list(volstring) == []:
        vol = 0
        vols.append(vol)
        return vols
    else:
        vol = int(volstring)
        vols.append(vol)
        return vols
    
    print("oopsies", volstring)
    return vols

def cleanData(files):
    dates = []
    spots = []
    vols = []

    for i in range(1,len(files)):
        row = files[i]
        rowlist = row.split(',')
        if rowlist != ['', '', '', '', '']:
            dates = getDates(rowlist, dates)
            spots.append(float(rowlist[2]))
            vols = getVols(rowlist,vols)
    
    return dates, spots, vols

In [40]:
#hp
hp_path = "../datafiles/numerical-initial/hp-data.csv"
hp_files = []
for hp_file in open(hp_path,"r"):
    hp_files.append(hp_file.split('\n')[0])
hp_dates, hp_spots, hp_vols = cleanData(hp_files)  

#ibm
ibm_path = "../datafiles/numerical-initial/ibm-data.csv"
ibm_files = []
for ibm_file in open(ibm_path,"r"):
    ibm_files.append(ibm_file.split('\n')[0])
ibm_dates, ibm_spots, ibm_vols = cleanData(ibm_files)

#dell
dell_path = "../datafiles/numerical-initial/dell-data.csv"
dell_files = []
for dell_file in open(dell_path,"r"):
    dell_files.append(dell_file.split('\n')[0])
dell_dates, dell_spots, dell_vols = cleanData(dell_files)

#seagate
seagate_path = "../datafiles/numerical-initial/seagate-data.csv"
seagate_files = []
for seagate_file in open(seagate_path,"r"):
    seagate_files.append(seagate_file.split('\n')[0])
seagate_dates, seagate_spots, seagate_vols = cleanData(seagate_files)

#westdig
westdig_path = "../datafiles/numerical-initial/westdig-data.csv"
westdig_files = []
for westdig_file in open(westdig_path,"r"):
    westdig_files.append(westdig_file.split('\n')[0])
westdig_dates, westdig_spots, westdig_vols = cleanData(westdig_files)


print(min(westdig_spots))
print(min(ibm_spots))
print(max(westdig_spots))
print(max(ibm_spots))


ibm_df = pd.DataFrame({'dates': ibm_dates, 'IBM spots': ibm_spots, 'IBM vols': ibm_vols})
ibm_df.head()

dell_df = pd.DataFrame({'dates': dell_dates, 'DELL spots': dell_spots, 'DELL vols': dell_vols})
dell_df.head()

hp_df = pd.DataFrame({'dates': hp_dates, 'HP spots': hp_spots, 'HP vols': hp_vols})
hp_df.head()

westdig_df = pd.DataFrame({'dates': westdig_dates, 'WESTDIG spots': westdig_spots, 'WESTDIG vols': westdig_vols})
westdig_df.head()

seagate_df = pd.DataFrame({'dates': seagate_dates, 'SEAGATE spots': seagate_spots, 'SEAGATE vols': seagate_vols})
seagate_df.head()

total_df = pd.merge(ibm_df, dell_df, on="dates")
total_df = pd.merge(total_df, hp_df, on="dates")
total_df = pd.merge(total_df, westdig_df, on="dates")
total_df = pd.merge(total_df, seagate_df, on="dates")

pd.DataFrame.to_csv(total_df, "./cleanedNumData.csv")

2.05
48.0625
114.28
215.8


In [33]:
ibm_df.head()

,IBM spots,IBM vols,dates
0,157.35,5481000.0,2018-03-19
1,160.26,9263000.0,2018-03-16
2,159.61,3871000.0,2018-03-15
3,158.12,3632000.0,2018-03-14
4,159.32,4186000.0,2018-03-13


In [34]:
total_df.head()

,IBM spots,IBM vols,dates,DELL spots,DELL vols,HP spots,HP vols,WESTDIG spots,WESTDIG vols,SEAGATE spots,SEAGATE vols
0,157.35,5481000.0,2018-03-19,77.97,1152000.0,23.20,9930000.0,103.16,4434000.0,60.40,6849000.0
1,160.26,9263000.0,2018-03-16,77.96,1228000.0,23.51,20053000.0,106.45,7290000.0,60.09,7073000.0
2,159.61,3871000.0,2018-03-15,77.62,1151000.0,23.48,10320000.0,102.23,2696000.0,59.57,2882000.0
3,158.12,3632000.0,2018-03-14,77.82,920749.0,23.48,9305000.0,103.20,3976000.0,59.93,4136000.0
4,159.32,4186000.0,2018-03-13,77.34,1421000.0,23.62,16956000.0,102.91,6893000.0,59.68,4375000.0
